## Generate text with **GPT** model

## Import libraries

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 5.1 MB/s 
     |████████████████████████████████| 6.6 MB 53.5 MB/s 
     |████████████████████████████████| 880 kB 63.9 MB/s 
     |████████████████████████████████| 77 kB 6.3 MB/s 
     |████████████████████████████████| 596 kB 61.4 MB/s 
     |████████████████████████████████| 96 kB 6.7 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.50-py3-none-any.whl size=895166 sha256=94c15dcf3091a154aa75c053a00defeb4f61ce73dc088139ddadee7d4839281f
  Stored in directory: /root/.cache/pip/wheels/d9/72/54/519f0d5143cc6c73fa3297509123c86fc8586a7fdea8d25311
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: click
    Found existing installation: click 7.1.2
    Uninstalling click-7.1.2:
      Successfully uninstalled click-7.1.2
ERROR: pip's dependency resolver does not currently take into a

In [25]:
import numpy as np
import pandas as pd
import re
import random

import torch
from tqdm.notebook import tqdm
import transformers

from transformers import GPT2LMHeadModel, AdamW

if torch.cuda.is_available():    
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [5]:
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('sberbank-ai/rugpt3small_based_on_gpt2')

Downloading:   0%|          | 0.00/1.63M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/608 [00:00<?, ?B/s]

## Open prepared file tokenize it

In [7]:
with open('./congr_text.txt', encoding='utf8') as f:
    text = f.read()

text = re.sub('\n{2,}', '\n', text)
print(text[:300])

Ты дочь востока, нежная Айгуль,
И солнышком в душе всегда согрета…
Ты жаркая, как будто бы июль,
Ты на земле, как яркий лучик света!
Хочу я в этот праздник для тебя
Дарить стихи и нежные признанья!
И поздравленья, искренне любя,
Зачесть в момент прекрасного свиданья!
Желаю красоту свою хранить
И мол


In [8]:
text

'Ты дочь востока, нежная Айгуль,\nИ солнышком в душе всегда согрета…\nТы жаркая, как будто бы июль,\nТы на земле, как яркий лучик света!\nХочу я в этот праздник для тебя\nДарить стихи и нежные признанья!\nИ поздравленья, искренне любя,\nЗачесть в момент прекрасного свиданья!\nЖелаю красоту свою хранить\nИ молодостью вечно наслаждаться!\nВсегда, во всем и всем довольной быть\nИ самой лучшей в мире оставаться!\n Айгуль, подружка верная моя,\nХочу поздравить с праздником тебя!\nСквозь суету давно прожитых лет\nПробьется пусть отрады яркий свет!\nХочу желать любви и процветанья,\nБольших доходов, радостей больших!\nИ от мужчин огромного вниманья\nИ достиженья всех земных вершин!\nТы оставайся навсегда такою\nПрекрасной, нежной, радостной, младой!\nЯ так горда, что век дружу с тобою!\nИ тем, что дружишь так же ты со мной!\n Своей, Айгуль, ты слепа красотой,\nМила, щедра и радостна безмерно!\nИ в этот праздник очень важный твой\nХочу желать я счастья без размера!\nТы утром просыпайся без оби

In [9]:
def cleansymbols(text):
    text = re.sub('\n', ' ', text)
    return text

In [10]:
text = cleansymbols(text)
text = text.strip()

In [11]:
text

'Ты дочь востока, нежная Айгуль, И солнышком в душе всегда согрета… Ты жаркая, как будто бы июль, Ты на земле, как яркий лучик света! Хочу я в этот праздник для тебя Дарить стихи и нежные признанья! И поздравленья, искренне любя, Зачесть в момент прекрасного свиданья! Желаю красоту свою хранить И молодостью вечно наслаждаться! Всегда, во всем и всем довольной быть И самой лучшей в мире оставаться!  Айгуль, подружка верная моя, Хочу поздравить с праздником тебя! Сквозь суету давно прожитых лет Пробьется пусть отрады яркий свет! Хочу желать любви и процветанья, Больших доходов, радостей больших! И от мужчин огромного вниманья И достиженья всех земных вершин! Ты оставайся навсегда такою Прекрасной, нежной, радостной, младой! Я так горда, что век дружу с тобою! И тем, что дружишь так же ты со мной!  Своей, Айгуль, ты слепа красотой, Мила, щедра и радостна безмерно! И в этот праздник очень важный твой Хочу желать я счастья без размера! Ты утром просыпайся без обид, Улыбкой солнце повстречат

In [12]:
tokens = tokenizer.encode(text, add_special_tokens=True)
tokens = np.array(tokens)
print(len(tokens))
tokens[:10]

147211


array([ 8804,  6304, 34619,    16, 49899,  7070, 49714,    16,   459,
        1679])

In [13]:
l = len(tokens)//15
train = []
test = []
for i in range(15):
    if i%5 > 0:
        train.extend(tokens[i*l: (i+1)*l])
    else:
        test.extend(tokens[i*l: (i+1)*l])
train = np.array(train)
test = np.array(test)

print(len(tokens), len(train), len(test))

147211 117768 29442


## Model

In [14]:
model = GPT2LMHeadModel.from_pretrained(
    'sberbank-ai/rugpt3small_based_on_gpt2',
    output_attentions = False,
    output_hidden_states = False,
)

model.to(device);

Downloading:   0%|          | 0.00/526M [00:00<?, ?B/s]

In [15]:
batch_size = 8
max_len = 256
epochs = 2

n_train = len(train)//(batch_size*max_len)
n_test = len(test)//(batch_size*max_len)
print(n_train, n_test)

optimizer = AdamW(model.parameters(), lr = 1e-5, eps = 1e-8)

total_steps = n_train * epochs
scheduler = transformers.get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, 
                                            num_training_steps = total_steps)


def accuracy(y_true, logits):
    return torch.mean((y_true[1:] == torch.argmax(logits, dim=2)[:-1]).float()).detach().cpu().numpy()

57 14


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [16]:
def prep_tensors(x, i, batch_size=batch_size, max_len=max_len):
    batch_ids = x[i*batch_size*max_len: (i+1)*batch_size*max_len]
    batch_ids = batch_ids.reshape(batch_size, max_len)
    batch_ids = torch.tensor(batch_ids).to(device)
    return batch_ids


for epoch in range(1, epochs+1):
    print(f'epoch {epoch}/{epochs} : training')

    train_loss = []
    train_acc = []
    model.train()
    pbar = tqdm(range(n_train))
    for i in pbar:
        batch_ids = prep_tensors(train, i)

        model.zero_grad()
        loss, logits, _ = model(batch_ids,
                             token_type_ids=None, 
                             labels=batch_ids
                             ).values()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        
        train_loss.append(loss.item())
        train_acc.append(accuracy(batch_ids, logits))
        pbar.set_description(f'acc {np.mean(train_acc):.4f} loss {np.mean(train_loss):.4f}', refresh=True)

    
    print(f'epoch {epoch}/{epochs} : validation')
    model.eval()
    val_acc = []
    val_loss = []
    pbar = tqdm(range(n_test))
    for i in pbar:
        batch_ids = prep_tensors(test, i)
        with torch.no_grad():        
            loss, logits, _ = model(batch_ids, 
                                token_type_ids=None, 
                                labels=batch_ids
                                 ).values()
        
        val_loss.append(loss.item())
        val_acc.append(accuracy(batch_ids, logits))
        pbar.set_description(f'acc {np.mean(val_acc):.4f} loss {np.mean(val_loss):.4f}', refresh=True)


epoch 1/2 : training


  0%|          | 0/57 [00:00<?, ?it/s]

epoch 1/2 : validation


  0%|          | 0/14 [00:00<?, ?it/s]

epoch 2/2 : training


  0%|          | 0/57 [00:00<?, ?it/s]

epoch 2/2 : validation


  0%|          | 0/14 [00:00<?, ?it/s]

In [17]:
# https://huggingface.co/transformers/main_classes/model.html#transformers.generation_utils.GenerationMixin.generate
# Model without training
import textwrap

prompt = 'Дорогая Маша'
prompt = tokenizer.encode(prompt, return_tensors='pt').to(device)
out = model.generate(
    input_ids=prompt,
    max_length=50,
    num_beams=5,
    do_sample=True,
    temperature=10.,
    top_k=50,
    top_p=0.6,
    no_repeat_ngram_size=3,
    num_return_sequences=3,
    ).cpu().numpy()
for out_ in out:
    print(textwrap.fill(tokenizer.decode(out_), 120), end='\n------------------\n')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Дорогая Маша!!!!!  Не забывай всегда меня поздравить  Дорогой, мой, добрый ангел!!! И желаю всегда любви и мира!!!! А
ты, я смотрю очень милый!  А с праздником поздравляю я тебя  Ты, милый и славный мужчина
------------------
Дорогая Маша!» А я все о той девочке мечтал, Чтобы в День рождения мы вдвоем гуляли и на танцы ездили! Ведь ты для меня
очень большая стала! А я желаю тебя все же порадовать: Заслужить заслуженно любовь! Тебя сегодня я
------------------
Дорогая Маша!  Сегодня утром меня на руках принесли И в путь повели на свидание Даришь радость в праздник милый и в беде
Ведь счастье для тебя на двоих Я на праздник иду без слов любви Чтобы все хорошее и
------------------


In [18]:
import textwrap

prompt = 'Дорогая Маша'
prompt = tokenizer.encode(prompt, return_tensors='pt').to(device)
out = model.generate(
    input_ids=prompt,
    max_length=50,
    num_beams=5,
    do_sample=True,
    temperature=5.,
    top_k=50,
    top_p=0.6,
    no_repeat_ngram_size=5,
    num_return_sequences=3,
    ).cpu().numpy()
for out_ in out:
    print(textwrap.fill(tokenizer.decode(out_), 120), end='\n------------------\n')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Дорогая Маша! С праздником! В твои года желаю В тебе не ошибиться. Хочу быть счастливой! Быть твоей лучшей женой И чтобы
с тобой вместе было тепло. Чтобы ты никогда не думала про грусти, И чтобы все было очень классно!
------------------
Дорогая Маша, В день рождения, ты очень милая В этот миг я тебя не узнал Я сегодня не с подарком был, Мне не до этого, я
жду Ближе к полуночи! Я от любви тебя не снимала,
------------------
Дорогая Маша, ты такая, как все мы: Ты добрая и ты - ангел! Будь ты счастлива, родная моя, На сердце твоем будет светло!
Милей и нежна я, Как ты мне очень дорога. И, как солнце светит,
------------------


In [175]:
def generate(prompt, len_gen=20, temperature=1):
    generated = tokenizer.encode(prompt)
    context = torch.tensor([generated]).to(device)
    past = None

    for i in tqdm(range(len_gen)):
        output, past = model(context, past_key_values=past).values()
        output = output / temperature
        token = torch.distributions.Categorical(logits=output[..., -1, :]).sample()
        
        generated += token.tolist()
        context = token.unsqueeze(0)

    sequence = tokenizer.decode(generated)

    sequence=re.sub(r'!',r'!\n', sequence)

    return sequence[:sequence.rfind('!')+1]

In [176]:
print(generate('Дорогая Маша!', len_gen=100))

  0%|          | 0/100 [00:00<?, ?it/s]

Дорогая Маша!
 Ты словно красная мармеладка!
 И для родных, и для близких Ты просто самая красивая!
 Пусть из этой роскоши ты не отстанешь!
 И не будешь скучать по компоту, Что всю жизнь ты пил!
  Машенька, сегодня ты чудесна и неотразима!
 Пусть день рожденья у тебя будет красивым, А судьба пусть тебя обязательно поздравит!
 Слушать тебя будет пусть Папа Римский, А он для тебя такой самый нежный!


In [23]:
torch.save(model, 'model.pt')